<a href="https://colab.research.google.com/github/mm230/NLP_Lab1_Week1/blob/main/Mansour_Sow_NLP_Lab1_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%cd NLP_Week_1_Labs_2022\ \(1\)/session1/

/content/drive/MyDrive/NLP_Week_1_Labs_2022 (1)/session1


In [9]:
%ls

logistic_regression.ipynb                       train1.txt  valid2.txt
Mansour_Sow_NLP_Lab1_logistic_regression.ipynb  train2.txt
naive_bayes.ipynb                               valid1.txt


In [1]:
import io, sys, math
import numpy as np
from collections import defaultdict

This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents. 

In [10]:
def build_dict(filename, threshold=1):
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [11]:
def load_data(filename, word_dict, label_dict):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    return data

First, let's implement the softmax function. Don't forget numerical stability!

In [12]:
def softmax(x):
    ### FILL CODE
    z = x - np.max(x)
    print(z)
    return np.exp(z) / np.sum(np.exp(z))

In [13]:
x = np.array([2, 100, 4])
print(softmax(x))

[-98   0 -96]
[2.74878501e-43 1.00000000e+00 2.03109266e-42]


Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

In [20]:
 def sgd(w, data, niter):
  nlabels, dim = w.shape
  learning_rate = 0.1
    
  for iter in range(niter):
    ### FILL CODE
      loss=0.0
      np.random.shuffle(data)
      for y, x in data:
        y_pred= softmax(np.dot(w, x))
        loss += math.log(y_pred[y])
        k = np.zeros(nlabels)
        k[y] = 1.0
        log_fi = y_pred - k
        gradient= log_fi.reshape(nlabels, 1) * x.reshape((1, dim))
        w -= learning_rate * gradient
      print("=" * 25)
      print('| ieration {:2d}| Loss: {:.3f}'.format(iter, loss))

  return w

The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [21]:
def predict(w, x):
    ## FILL CODE
    prediction = softmax(np.dot(w, x))
    return np.argmax(prediction)

Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [22]:
def compute_accuracy(w, valid_data):
    ## FILL CODE
    n = len(valid_data)
    good_prediction = 0.0
    for y, x in valid_data:
      prediction = predict(w, x)
      if prediction == y:
        good_prediction += 1 
    return good_prediction / n * 100.0

In [23]:
print("="* 40)
print("** Logistic Regression **")
print("="*40)

word_dict, label_dict = build_dict("train1.txt")
train_data = load_data("train1.txt", word_dict, label_dict)
valid_data = load_data("valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)
dim = len(word_dict)
w = np.zeros([nlabels, dim])
w = sgd(w, train_data, 5)
print("="*40)
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("="*40)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
 -4.37469604 -4.37489022 -4.37485109 -4.37475071]
[-6.60645046 -6.75787007 -6.5396136  -2.69109138 -6.71447785 -2.03220782
 -6.94026503  0.         -7.20207529 -4.30263604]
[-8.25940831 -8.26079973 -8.10745628 -8.41556109  0.         -8.19040684
 -8.34266674 -8.12170735 -8.20618976 -8.17382484]
[-10.40342027 -10.32547734 -10.20749026  -5.02083211 -10.70861929
  -4.90829276  -9.0385388    0.          -7.76659718  -5.36420409]
[-8.74889099 -8.86135083 -8.69491548 -5.61489295 -8.83482013 -6.06460743
 -8.96187798  0.         -9.14194154 -6.49962663]
[-14.44277423 -14.51850478 -14.09916637 -14.68498358   0.
 -12.96052945 -14.7504776  -13.90154413 -13.15754114 -14.3055916 ]
[-6.82378844 -6.82376441  0.         -6.82381509 -6.82381776 -6.82381178
 -6.82381624 -6.82380294 -6.8238082  -6.82381208]
[-1.04473806 -1.99927217 -1.99221977 -2.05094208 -1.9859815  -1.61922007
 -1.9904697  -2.14041718 -2.03990826  0.        ]